In [1]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# Sample dataset
data = '/Users/andreybalabin/Desktop/Sean_Gomez_Lab/klaeger.csv'
df = pd.read_csv(data)

# Initialize the Dash app
app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    html.Label("Select Concentration:"),
    dcc.Dropdown(
        id='concentration-dropdown',
        options=[{'label': conc, 'value': conc} for conc in df['concentration_M'].unique()],
        value=df['concentration_M'].unique()[0]
    ),
    
    html.Div([
        html.Button('Drug', id='drug-button', n_clicks=0, style={'marginRight': '10px'}),
        html.Button('Gene', id='gene-button', n_clicks=0)
    ]),
    
    dcc.Dropdown(id='drug-gene-dropdown', style={'display': 'none'}),

    dcc.Graph(id='bar-chart'),
    dcc.Graph(id='second-bar-chart', style={'display': 'none'})
])

# Define callback to update dropdown options and visibility
@app.callback(
    [Output('drug-gene-dropdown', 'options'),
     Output('drug-gene-dropdown', 'value'),
     Output('drug-gene-dropdown', 'style')],
    [Input('drug-button', 'n_clicks'),
     Input('gene-button', 'n_clicks'),
     Input('concentration-dropdown', 'value')]
)
def update_dropdown(drug_clicks, gene_clicks, selected_concentration):
    # Filter dataframe based on selected concentration
    filtered_df = df[df['concentration_M'] == selected_concentration]

    # Determine which button was clicked
    is_drug_button = drug_clicks % 2 == 1
    is_gene_button = gene_clicks % 2 == 1

    # Get unique values and dropdown visibility based on the selected column
    if is_drug_button:
        column_values = filtered_df['drug']
        dropdown_style = {'display': 'block'}
    elif is_gene_button:
        column_values = filtered_df['gene_name']
        dropdown_style = {'display': 'block'}
    else:
        # Default to drug column and hide dropdown if neither button is clicked
        column_values = filtered_df['drug']
        dropdown_style = {'display': 'none'}

    options = [{'label': val, 'value': val} for val in column_values.unique()]

    return options, options[0]['value'], dropdown_style

# Define callback to update chart based on selected concentration and dropdown value
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('concentration-dropdown', 'value'),
     Input('drug-gene-dropdown', 'value')]
)
def update_chart(selected_concentration, selected_value):
    # Filter dataframe based on selected concentration and dropdown value
    filtered_df = df[(df['concentration_M'] == selected_concentration) & 
                     ((df['drug'] == selected_value) | (df['gene_name'] == selected_value)) &
                     (df['relative_intensity'] < 1)]  # Exclude values 1 or above

    # Sort the dataframe by inhibition values
    filtered_df = filtered_df.sort_values(by='relative_intensity')

    # Create the bar chart with the updated title
    fig = px.bar(filtered_df, x='drug' if 'drug' in filtered_df else 'gene_name',
                 y='relative_intensity', title=f'Inhibition of {selected_value} at {selected_concentration} M')

    # Enable click data for the first bar chart
    fig.update_traces(marker_color='lightblue', selector=dict(type='bar'))

    return fig

# Define callback to update second chart based on selected column in the first chart
@app.callback(
    [Output('second-bar-chart', 'figure'),
     Output('second-bar-chart', 'style')],
    [Input('bar-chart', 'clickData'),
     Input('concentration-dropdown', 'value')]
)
def update_second_chart(click_data, selected_concentration):
    if click_data is None:
        # If no column is clicked, hide the second chart
        return {}, {'display': 'none'}
    
    # Get the selected column value from the clicked data
    selected_column_value = click_data['points'][0]['x']

    # Determine whether the selected column is a gene or a drug
    is_gene_column = 'gene_name' in df.columns and selected_column_value in df['gene_name'].unique()
    is_drug_column = 'drug' in df.columns and selected_column_value in df['drug'].unique()

    # Filter dataframe based on selected concentration and clicked column value
    if is_gene_column:
        # Show drug inhibition values for the selected gene
        filtered_df = df[(df['concentration_M'] == selected_concentration) & 
                         (df['gene_name'] == selected_column_value) &
                         (df['relative_intensity'] < 1)]
        x_label = 'drug'
        chart_title = f'Drug Inhibition for Gene: {selected_column_value}'
    elif is_drug_column:
        # Show gene inhibition values for the selected drug
        filtered_df = df[(df['concentration_M'] == selected_concentration) & 
                         (df['drug'] == selected_column_value) &
                         (df['relative_intensity'] < 1)]
        x_label = 'gene_name'
        chart_title = f'Gene Inhibition for Drug: {selected_column_value}'
    else:
        # If neither gene nor drug, return empty figure
        return {}, {'display': 'none'}

    # Create the second bar chart
    second_fig = px.bar(filtered_df, x=x_label, y='relative_intensity', title=chart_title)

    return second_fig, {'display': 'block'}

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
